In [1]:
# Load the model and dataset through the controller
from Controller.PredictingController import PredictingController
predictor = PredictingController()


| Preprocessed data loaded successfully 

| Model loaded successfully 


<Figure size 1600x800 with 0 Axes>

In [3]:
## show the training dataset head
predictor.train_dataset.dataset.df.head(5)

,trace,caseid
0,"[2, 12, 9, 10, 27, 28, 3, 17, 8, 15, 19, 33, 2...",173688
1,"[2, 12, 9, 10, 27, 28, 26, 28, 3, 17, 8, 15, 1...",173691
2,"[2, 12, 9, 10, 27, 28, 26, 28, 26, 28, 26, 28,...",173694
3,"[2, 12, 9, 7, 1]",173697
4,"[2, 12, 9, 7, 1]",173700


In [5]:
## Show the dict
predictor.dataset.index_to_vocab(2)

'<SOS>'

In [9]:
## Print vocaburaries with corresponded index.
predictor.dataset._BPI2012Dataset__vocab_dict

{'<EOS>': 1,
 '<SOS>': 2,
 'A_ACCEPTED_COMPLETE': 3,
 'A_ACTIVATED_COMPLETE': 4,
 'A_APPROVED_COMPLETE': 5,
 'A_CANCELLED_COMPLETE': 6,
 'A_DECLINED_COMPLETE': 7,
 'A_FINALIZED_COMPLETE': 8,
 'A_PARTLYSUBMITTED_COMPLETE': 9,
 'A_PREACCEPTED_COMPLETE': 10,
 'A_REGISTERED_COMPLETE': 11,
 'A_SUBMITTED_COMPLETE': 12,
 'O_ACCEPTED_COMPLETE': 13,
 'O_CANCELLED_COMPLETE': 14,
 'O_CREATED_COMPLETE': 15,
 'O_DECLINED_COMPLETE': 16,
 'O_SELECTED_COMPLETE': 17,
 'O_SENT_BACK_COMPLETE': 18,
 'O_SENT_COMPLETE': 19,
 'W_Afhandelen leads_COMPLETE': 20,
 'W_Afhandelen leads_SCHEDULE': 21,
 'W_Afhandelen leads_START': 22,
 'W_Beoordelen fraude_COMPLETE': 23,
 'W_Beoordelen fraude_SCHEDULE': 24,
 'W_Beoordelen fraude_START': 25,
 'W_Completeren aanvraag_COMPLETE': 26,
 'W_Completeren aanvraag_SCHEDULE': 27,
 'W_Completeren aanvraag_START': 28,
 'W_Nabellen incomplete dossiers_COMPLETE': 29,
 'W_Nabellen incomplete dossiers_SCHEDULE': 30,
 'W_Nabellen incomplete dossiers_START': 31,
 'W_Nabellen offertes

In [11]:
## Evaluation on training and test sets.abs
predictor.perform_eval_on_training_set()


| Performing Evaluation on whole dataset ... 
/home/jrhs/Documents/GitHub/Apply-LINDA-BN-on-predicting-next-event/Data/BPI2012Dataset.py:256: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  sorted_seq_list = [torch.tensor(seq_list[idx])

| Evaluation result | Accuracy [0.8532] | Loss [3.4207] 


In [13]:
predictor.perform_eval_on_testing_set()


| Performing Evaluation on whole dataset ... 
/home/jrhs/Documents/GitHub/Apply-LINDA-BN-on-predicting-next-event/Data/BPI2012Dataset.py:256: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  sorted_seq_list = [torch.tensor(seq_list[idx])

| Evaluation result | Accuracy [0.8526] | Loss [3.4246] 


In [ ]:
    ## 